# Reading CSV File

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas matplotlib

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import explode, col


sc = SparkContext('local', 'Spark SQL') 
spark = SQLContext(sc)

In [3]:
df = (spark.read
            .option("header", "true")
            .option("delimiter", ";")
            .option("mode", "DROPMALFORMED")
            .option("inferschema", "true")
            .option("encoding", "ISO-8859-1")
            .csv("2019-2-semestre.csv"))

In [4]:
df.printSchema()

root
 |-- Região: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Faixa Etária: string (nullable = true)
 |-- Data Finalização: string (nullable = true)
 |-- Tempo Resposta: integer (nullable = true)
 |-- Nome Fantasia: string (nullable = true)
 |-- Segmento de Mercado: string (nullable = true)
 |-- Área: string (nullable = true)
 |-- Assunto: string (nullable = true)
 |-- Grupo Problema: string (nullable = true)
 |-- Problema: string (nullable = true)
 |-- Como Comprou Contratou: string (nullable = true)
 |-- Procurou Empresa: string (nullable = true)
 |-- Respondida: string (nullable = true)
 |-- Situação: string (nullable = true)
 |-- Avaliação Reclamação: string (nullable = true)
 |-- Nota do Consumidor: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [5]:
df.show(5, False)

+------+---+--------------+----+------------------+----------------+--------------+-----------------------+-----------------------------------------------------------------------+-----------------------------------+--------------------------------------------------------+----------------------+----------------------------------------------------------------------------------+-----------------------+----------------+----------+-----------------------+--------------------+------------------+-----+
|Região|UF |Cidade        |Sexo|Faixa Etária      |Data Finalização|Tempo Resposta|Nome Fantasia          |Segmento de Mercado                                                    |Área                               |Assunto                                                 |Grupo Problema        |Problema                                                                          |Como Comprou Contratou |Procurou Empresa|Respondida|Situação               |Avaliação Reclamação|Nota do Consumidor|Total

In [6]:
df.registerTempTable("consumidor")

In [7]:
spark.sql("""
select `Região`, count(*) 
from consumidor
group by `Região`
""").show()

+------+--------+
|Região|count(1)|
+------+--------+
|    NE|   75340|
|    S |   89456|
|    SE|  196927|
|    N |   14869|
|    CO|   46799|
+------+--------+



In [11]:
df.limit(5).toPandas()

,Região,UF,Cidade,Sexo,Faixa Etária,Data Finalização,Tempo Resposta,Nome Fantasia,Segmento de Mercado,Área,Assunto,Grupo Problema,Problema,Como Comprou Contratou,Procurou Empresa,Respondida,Situação,Avaliação Reclamação,Nota do Consumidor,Total
0,NE,PE,Petrolina,M,entre 31 a 40 anos,05/07/2019,10.0,Banco Bradesco,"Bancos, Financeiras e Administradoras de Cartão",Serviços Financeiros,Conta corrente / Salário / Poupança /Conta Apo...,Cobrança / Contestação,"Cobrança submetendo a ofensa, constrangimento ...",Domicílio,S,S,Finalizada não avaliada,Não Avaliada,NaN,1
1,SE,SP,São Carlos,F,entre 51 a 60 anos,03/07/2019,NaN,Azul Linhas Aéreas,Transporte Aéreo,Transportes,Cargas e Encomendas,Vício de Qualidade,"Extravio/ avaria de bagagem, carga, correspond...",Loja física,S,N,Finalizada não avaliada,Não Avaliada,NaN,1
2,SE,SP,São Paulo,M,entre 31 a 40 anos,03/07/2019,NaN,Samsung,"Fabricantes - Eletroeletrônicos, Produtos de ...",Produtos de Telefonia e Informática,Aparelho celular,Entrega do Produto,Produto entregue incompleto / diferente do pe...,Ganhei de presente,S,N,Finalizada não avaliada,Não Avaliada,NaN,1
3,SE,RJ,Rio de Janeiro,F,entre 61 a 70 anos,02/07/2019,10.0,Tim,"Operadoras de Telecomunicações (Telefonia, Int...",Produtos de Telefonia e Informática,Aparelho celular,Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Internet,S,S,Finalizada não avaliada,Não Avaliada,NaN,1
4,NE,PE,Recife,M,entre 21 a 30 anos,04/07/2019,10.0,Vivo - Telefônica (GVT),"Operadoras de Telecomunicações (Telefonia, Int...",Telecomunicações,Internet Fixa,Vício de Qualidade,Suspensão ou desligamento indevido do serviço,Não comprei / contratei,S,S,Finalizada avaliada,Não Resolvida,1.0,1


In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

In [ ]:
def to_value(v):
    try:
        return float(v.replace(",","."))